# Data Exploration
## Libraries import and function Definitions

In [ ]:
%load_ext autoreload
%autoreload 2

import sqlalchemy
import pandas as pd
import missingno as msno 
import matplotlib.pyplot as plt
import scipy
# %matplotlib widget

import seaborn as sns
import numpy as np
from sklearn.metrics import mean_squared_error,root_mean_squared_error,mean_absolute_error,r2_score
import matplotlib.ticker as mtick
from CustomLibs.CustomFunctions import plot_corr_heatmap, value_to_float, fig_indexes, sqlcol
from config import Config
from sklearn.model_selection import TimeSeriesSplit

date_val_end=Config.TEST_DATE_CUTOFF
date_test_start=pd.to_datetime(date_val_end) + pd.DateOffset(days=1)

from joypy import joyplot

engine = sqlalchemy.create_engine(Config.CONN_STR)




## Load Data and Type Setting

In [ ]:
with engine.connect() as conn:
    # Import leading variables - attributes which are known ahead of time
    with open('.\SQL Files\CombineAllRawFeatures.sql', 'r') as query:
        df_all = pd.read_sql_query(query.read(),conn)
df_all['Date']=pd.to_datetime(df_all['Date'])
df_all['School_Holiday']=df_all['School_Holiday'].apply(bool).astype(bool)
df_all['Meeting_Participants']=df_all['Meeting_Participants'].apply(value_to_float).astype(float)
df_all.head()


with engine.connect() as conn:
    df_all.to_sql('All_Raw_Features',conn,schema='Silver',if_exists='replace',index=False,dtype=sqlcol(df_all))


# df_electric=df_all[['Day_Electric_KWh', 'Night_Electric_KWh']]
drop_list = [x for x in df_all.columns if x.endswith('_ori')]
print(drop_list)
val_list = [x.replace('_ori','') for x in drop_list]
print(val_list)
df_all.drop(columns=drop_list,inplace=True)
df_all.rename(dict(zip(drop_list, val_list)),axis=1,inplace=True)

# df_all.drop(columns=['Day_Electric_KWh', 'Night_Electric_KWh', 'Heat_Consumption', 'Cold_Consumption'],inplace=True)
# df_all.rename({'Day_Electric_KWh_ori':'Day_Electric_KWh',
#                'Night_Electric_KWh_ori':'Night_Electric_KWh',
#                'Heat_Consumption_ori':'Heat_Consumption',
#                'Cold_Consumption_ori':'Cold_Consumption'},axis=1,inplace=True)

print(f'loaded {len(df_all.columns)} columns and {len(df_all)} rows')




# leading_features=df_all.columns[:13].tolist()
# lagging_features=df_all.columns[14:-1].tolist()
label_field = df_all.columns[-1]

# print(leading_features)
# print(lagging_features)
print(label_field)

## Date Exploration

### Datasets Missing Values

In [ ]:
ax1 = msno.matrix(df_all,figsize=(20,5))
ax1.plot()
ax1.figure.savefig('./Output Files/Images/Data Exploration/missing_values_matrix.png',format='png',bbox_inches='tight')

In [ ]:
ax2 = msno.bar(df_all,figsize=(16,6))
ax2.grid(visible=True,which='Major',axis='y') 
ax2.axhline(y=0.8,xmin=0,xmax=1, linewidth=2, color='r',linestyle='-',label='Cutoff')
# print(type(ax2))
# ax2.legend()
# ax2.plot()
# ax2.set_ylim(0,1.2)
# ax2.figure.tight_layout()
# ax2._secondary_axes.SecondaryAxis.tick_params(axis='both', which='major', labelsize=8)
# ax2.tick_params(axis='both', which='major', labelsize=8)
ax2.figure.savefig('./Output Files/Images/Data Exploration/missing_values_bar.png',format='png',bbox_inches='tight')

print(len(df_all))
print(len(df_all.dropna()))

print(len(df_all.set_index('Date').loc[:date_val_end]))
print(len(df_all.set_index('Date').loc[:date_val_end].dropna()))

### Raw Datasets .describe()

In [ ]:
df_all.describe().to_csv('./Output Files/Text/Data Exploration/feature_description.csv')
with open('./Output Files/Text/Data Exploration/feature_info.txt', 'w') as f:
    df_all.info(buf=f)

not_enough_data=[]
for columnName, columnData in df_all.items():
    if columnData.count() / len(columnData) < 0.8:
        not_enough_data.append(columnName)
print(not_enough_data)
df_all.drop(columns=not_enough_data,inplace=True)

# df_all.info()

leading_features=df_all.columns[:11].tolist()
lagging_features=df_all.columns[12:-1].tolist()
label = df_all.columns[-1]

print(leading_features)
print(lagging_features)
print(label)
# df_all.pivot(columns='Day_Name',values='Actual_Desks_Used').describe().to_csv('./Output Files/Text/Data Exploration/attendance_by_day_description.csv')

### Datasets .info()

### Attendence Distribution by Day

In [ ]:
fig,axs=plt.subplots(1,2,figsize=(10,4))

days_of_week=['Monday','Tuesday','Wednesday','Thursday','Friday']
df_all['Day']=df_all['Date'].dt.day_name()
# df_all['TestFlag']=df_all['Date']<=date_val_end
df_all['TestFlag'] = np.where(df_all['Date']<=date_val_end, 'Train / Validation', 'Test')
# df_all['Day_Number']=df_all['Date'].dt.day_of_week()


sns.histplot(data=df_all,x='Actual_Desks_Used',ax=axs[0],hue='TestFlag',common_norm=False,kde=True,bins=25,stat='percent')
axs[0].set_title(f'Distribution By Training and Test' ,fontsize=16)
axs[0].set_xlabel('Staff on Site',fontsize=12)


sns.histplot(data=df_all,x='Actual_Desks_Used',ax=axs[1],hue='Day',kde=True,bins=25,stat='percent',common_norm=True,element='step')
axs[1].set_title(f'Distribution By Day' ,fontsize=16)
axs[1].set_xlabel('Staff on Site',fontsize=12)

if Config.MASK_VALUE:
    axs[0].set_xticklabels([])
    axs[1].set_xticklabels([])
fig.tight_layout()
fig.savefig('./Output Files/Images/Data Exploration/Attendance_Distribution.png',format='png',bbox_inches='tight')

df_all.drop(columns='TestFlag',inplace=True)


In [ ]:
from matplotlib import cm

df_all['Day_Of_Week']=df_all['Date'].dt.day_of_week
# display(df_all[['Day_Of_Week','Day']].sort_values('Day_Of_Week'))
df_g=df_all.groupby('Day',sort=False)
fig, axes = joyplot(df_g, column='Actual_Desks_Used', colormap=cm.Pastel1, grid="y", overlap = 3, fade=False,title='Attendance Frequency Distribution By Day',linewidth=1,figsize=(6,5))

#     axes.set_xticklabels([])
for ax in axes:
    if Config.MASK_VALUE:
        ax.set_xticklabels([])
    ax.set_xlabel('Staff On Site')

fig.savefig('./Output Files/Images/Data Exploration/Attendance_Distribution_joyday.png',format='png',bbox_inches='tight')

In [ ]:
# df_g=df_all.groupby('Day',sort=False)
df_all['TestFlag'] = np.where(df_all['Date']<=date_val_end, 'Train / Validation', 'Test')
fig, axes = joyplot(df_all, by='TestFlag', column='Actual_Desks_Used', colormap=cm.Pastel1, grid="y", overlap = 1, fade=False,title='Attendance Frequency Distribution By Dataset',linewidth=1,figsize=(6,3))

#     axes.set_xticklabels([])
for ax in axes:
    if Config.MASK_VALUE:
        ax.set_xticklabels([])
    ax.set_xlabel('Staff On Site')

fig.savefig('./Output Files/Images/Data Exploration/Attendance_Distribution_joydataset.png',format='png',bbox_inches='tight')

### Attendance by Date and Day of Week

In [ ]:
fig1,ax1=plt.subplots(1,1,figsize=(10,4))
fig2,ax2=plt.subplots(1,1,figsize=(10,4))
axs=[ax1,ax2]
df_all['Pct_On_Site']=df_all['Actual_Desks_Used']/df_all['Total_Staff']
sns.lineplot(x='Date',y='Pct_On_Site',data = df_all,hue='Day',ax=axs[0])
axs[0].set_title(f'Attendance by Day',fontsize=16)

# sns.lineplot(x='Date',y='PctOnSite',data = df_leading,hue='DayName',ax=ax)
df_all['Week_Number']=df_all['Date'].dt.isocalendar().week
df_all['Year']=df_all['Date'].dt.year
df_byweek=df_all.groupby(['Week_Number','Year'],as_index=False).agg({'Date':'min','Actual_Desks_Used':'sum','Total_Staff':'sum','Total_Leave':'sum'})
df_byweek['Pct_On_Site']=df_byweek['Actual_Desks_Used']/df_byweek['Total_Staff']
df_byweek['Pct_On_Site_hols']=df_byweek['Actual_Desks_Used']/(df_byweek['Total_Staff']-df_byweek['Total_Leave'])


sns.lineplot(x='Date',y='Pct_On_Site',data = df_byweek,ax=axs[1],label='Weekly Total')
sns.lineplot(x='Date',y='Pct_On_Site_hols',data = df_byweek,ax=axs[1],label='Weekly Total (excl Leave)')
# axs[1].hlines(y=0.5,xmin=min(df_byweek['Date']),xmax=max(df_byweek['Date']), linewidth=1, color='r',linestyles='dashed',label='Target')
axs[1].set_title(f'Total Attendance per Week',fontsize=16)

for ax in axs:
    
    ax.set_ylabel('% Staff On-Site',fontsize=12)
    ax.set_xlabel('Date',fontsize=12)
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    ax.set_ylim(0,0.6)
    ax.grid(visible=True,which='Major',axis='both') 
    ax.legend(loc='upper left')
plt.tight_layout()

if Config.MASK_VALUE:
    axs[0].set_yticklabels([])
    axs[1].set_yticklabels([])

fig1.savefig('./Output Files/Images/Data Exploration/Attendance_PerDay_and_PerWeek.png',format='png',bbox_inches='tight')

plt.show()

## Correlation

In [ ]:
# df_silver=df_leading.merge(right=df.drop(columns=['PctOnSite','ma5','ma7','ma11']),how='left',on='Date')

# fig0,ax0=plt.subplots(figsize=(12,10))
fig1,ax1=plt.subplots(figsize=(10,10))
fig2,ax2=plt.subplots(figsize=(10,10))

args={'annot':False}

# plot_corr_heatmap(df=df_ts.select_dtypes(include=np.number),ax=ax0,**args)
droplist=['Week_Number','Year','Pct_On_Site','Car_Parking_Capacity','Motorbike_Parking_Capacity','Bike_Parking_Capacity']
plot_corr_heatmap(df=df_all.drop(columns=droplist).select_dtypes(include=np.number),ax=ax1)
ax1.tick_params(axis='both', which='major', labelsize=10)
ax1.set_title('Pearson r Correlation Matrix')

plt.tight_layout()
fig1.savefig('./Output Files/Images/Data Exploration/Feature_Correlation.png',format='png',bbox_inches='tight')


plot_corr_heatmap(df=df_all.drop(columns=droplist).select_dtypes(include=np.number),method='spearman',ax=ax2)
ax2.set_title('Spearman rho Correlation Matrix')
ax2.tick_params(axis='both', which='major', labelsize=10)
plt.tight_layout()
fig2.savefig('./Output Files/Images/Data Exploration/Feature_Correlation_spearman.png',format='png',bbox_inches='tight')

## Outliers
### Leading Attributes

In [ ]:
# print(len(df_leading.select_dtypes(include=np.number).columns))
fig1,axs1=plt.subplots(8,6,figsize=(8,12))
# fig1.subplots_adjust(wspace=2)

fig2,axs2=plt.subplots(12,4,figsize=(8,10))
# fig2.subplots_adjust(wspace=0.2,hspace=0.8)

# Iterate through attributes
dont_include=['Date','Desks_Booked','Cold_Consumption','Heat_Consumption','Car_Parking_Capacity', 'Motorbike_Parking_Capacity', 'Bike_Parking_Capacity','Week_Number','Year','Day','Total_Staff','School_Holiday']
['Total_Attendees', 'Total_Meetings', 'Air_Conditioning', 'East_Floor_Usage', 'West_Floor_Usage']
['Date', 'Desks_Booked', 'Total_Staff', 'Min_Air_Temp', 'Max_Air_Temp', 'Rainfall', 'Windspeed', 'School_Holiday', 'Annual_Leave', 'Flexi_Leave', 'Total_Leave']
['FTE_Count', 'Car_Parking_Occupancy', 'Car_Parking_Capacity', 'Motorbike_Parking_Occupancy', 'Motorbike_Parking_Capacity', 'Bike_Parking_Occupancy', 'Bike_Parking_Capacity', 'Daily_Transactions', 'Breakfast', 'Lunch', 'Hot_Meals', 'VPN_cnxn', 'Webex_Connections', 'Webex_Total_Participants', 'Webex_Maximum_Concurrent_Meetings', 'Total_Electric_KWh', 'Day_Electric_KWh', 'Night_Electric_KWh', 'Gas_Consumption', 'Kitchen_Usage', 'Water_Consumption', 'Cold_Consumption', 'Heat_Consumption', 'Total_Monthly_Biodegradable', 'Total_Monthly_Landfill', 'Total_Monthly_Recycable', 'Total_Monthly_Waste']




for i,(columnName, columnData) in enumerate(df_all.drop(columns=dont_include).items()):#.select_dtypes(include=np.number).drop(columns=droplist).items()):
    # Get x,y figure co-ords based on current i
    x,y=fig_indexes(6,i)
    try:
    # Plot results using seaborn histplot
        sns.boxplot(data=columnData,ax=axs1[x][y],
                        notch=False, showcaps=True,
                        flierprops={"marker": "x"},
                        boxprops={"facecolor": (.3, .5, .7, .5)},
                        medianprops={"color": "r", "linewidth": 2},)
        if Config.MASK_VALUE:
            axs1[x][y].set_yticklabels([])
        x,y=fig_indexes(4,i)
        sns.histplot(data=columnData,ax=axs2[x][y],stat='frequency',kde=True)
        if y>0:
            axs2[x][y].set(ylabel=None)
        axs2[x][y].set(xlabel=None)
        axs2[x][y].set_title(f'{columnName}',fontsize=10)

        if Config.MASK_VALUE:
            axs2[x][y].set_xticklabels([])
    except:
        pass

# Remove extra ax
for j in range(i+1,8*6):
    x,y=fig_indexes(6,j)
    fig1.delaxes(axs1[x,y])

for j in range(i+1,12*4):
    x,y=fig_indexes(4,j)
    fig2.delaxes(axs2[x,y])

fig1.tight_layout()
fig2.tight_layout()
fig1.savefig('./Output Files/Images/Data Exploration/Boxplots.png',format='png',bbox_inches='tight')
fig2.savefig('./Output Files/Images/Data Exploration/histplots.png',format='png',bbox_inches='tight')

### For report version

In [ ]:
# print(len(df_leading.select_dtypes(include=np.number).columns))
fig1,axs1=plt.subplots(1,6,figsize=(8,3))
# fig1.subplots_adjust(wspace=2)

fig2,axs2=plt.subplots(1,4,figsize=(8,2))
# fig2.subplots_adjust(wspace=0.2,hspace=0.8)

# Iterate through attributes
dont_include=['Date','Desks_Booked','Cold_Consumption','Heat_Consumption','Car_Parking_Capacity', 'Motorbike_Parking_Capacity', 'Bike_Parking_Capacity','Week_Number','Year','Day','Total_Staff','School_Holiday']
['Total_Attendees', 'Total_Meetings', 'Air_Conditioning', 'East_Floor_Usage', 'West_Floor_Usage']
['Date', 'Desks_Booked', 'Total_Staff', 'Min_Air_Temp', 'Max_Air_Temp', 'Rainfall', 'Windspeed', 'School_Holiday', 'Annual_Leave', 'Flexi_Leave', 'Total_Leave']
['FTE_Count', 'Car_Parking_Occupancy', 'Car_Parking_Capacity', 'Motorbike_Parking_Occupancy', 'Motorbike_Parking_Capacity', 'Bike_Parking_Occupancy', 'Bike_Parking_Capacity', 'Daily_Transactions', 'Breakfast', 'Lunch', 'Hot_Meals', 'VPN_cnxn', 'Webex_Connections', 'Webex_Total_Participants', 'Webex_Maximum_Concurrent_Meetings', 'Total_Electric_KWh', 'Day_Electric_KWh', 'Night_Electric_KWh', 'Gas_Consumption', 'Kitchen_Usage', 'Water_Consumption', 'Cold_Consumption', 'Heat_Consumption', 'Total_Monthly_Biodegradable', 'Total_Monthly_Landfill', 'Total_Monthly_Recycable', 'Total_Monthly_Waste']


do_include_box=['FTE_Count','Meeting_Duration','Rainfall','Pct_On_Site','Meeting_Cnxns','Car_Parking_Occupancy']
do_include_histo=['Hot_Meals','Day_Electric_KWh','Car_Parking_Occupancy','Annual_Leave']

for i,(columnName, columnData) in enumerate(df_all[do_include_box].items()):#.select_dtypes(include=np.number).drop(columns=droplist).items()):
    # Get x,y figure co-ords based on current i

    try:
    # Plot results using seaborn histplot
        sns.boxplot(data=columnData,ax=axs1[i],
                        notch=False, showcaps=True,
                        flierprops={"marker": "x"},
                        boxprops={"facecolor": (.3, .5, .7, .5)},
                        medianprops={"color": "r", "linewidth": 2},)
        if Config.MASK_VALUE:
            axs1[i].set_yticklabels([])
        

    except:
        pass
fig1.suptitle('Selected Box Plots',fontsize=12,fontweight='bold')


for i,(columnName, columnData) in enumerate(df_all[do_include_histo].items()):#.select_dtypes(include=np.number).drop(columns=droplist).items()):
    # Get x,y figure co-ords based on current i

    try:
    # Plot results using seaborn histplot
        sns.histplot(data=columnData,ax=axs2[i],stat='frequency',kde=True)
        if Config.MASK_VALUE:
            axs2[i].set_xticklabels([])
        

    except:
        pass
fig1.suptitle('Selected Box Plots',fontsize=12)
fig2.suptitle('Selected Histograms',fontsize=12)


fig1.tight_layout()
fig2.tight_layout()
fig1.savefig('./Output Files/Images/Data Exploration/Boxplots_onerow.png',format='png',bbox_inches='tight')
fig2.savefig('./Output Files/Images/Data Exploration/histplots_onerow.png',format='png',bbox_inches='tight')

In [ ]:
outlier_stats={}
outlier_list=[]

zthresh1=3.0
zthresh2=5.0
fig,axs=plt.subplots(20,1)

i=0
dates=df_all['Date']

for columnName,columnData in df_all.drop(columns=droplist).select_dtypes(include=np.number).items():
    zmean=np.mean(columnData.loc[lambda x : x != 0])
    zstd=np.std(columnData.loc[lambda x : x != 0])
    zdist = np.abs(columnData - zmean) / zstd
    mask1 = (zthresh1 < zdist) & (zthresh2 > zdist)
    mask2 = (zthresh2 <= zdist)

    if np.sum(mask1) + np.sum(mask2) > 0:
        # x,y=fig_indexes(2,i)
        
        zoutliers1 = 100*np.sum(mask1)/len(columnData)
        zoutliers2 = 100*np.sum(mask2)/len(columnData)
        outlier_stats[columnName]=[zmean,zstd,zoutliers1,zoutliers2,np.sum(mask1),np.sum(mask2),len(columnData),zthresh1,zthresh2]
        if np.sum(mask2) > 0:
            outlier_list.append(columnName)
            # axs[i].hlines(y=zmean,xmin=min(dates),xmax=max(dates), linewidth=0.5, color='r',linestyles='dashed',label='Mean excl. 0 values')
            # axs[i].hlines(y=zmean+zthresh2*zstd,xmin=min(dates),xmax=max(dates), linewidth=0.5, color='r',linestyles='dashed',label=f'{zthresh2} Z')
            # axs[i].hlines(y=zmean-zthresh2*zstd,xmin=min(dates),xmax=max(dates), linewidth=0.5, color='r',linestyles='dashed',label=f'-{zthresh2} Z')
            # axs[i].fill_between(dates, zmean - zthresh1*zstd, zmean + zthresh1*zstd, alpha=0.1,color='c',label=f'{zthresh1} x Std.Dev')
            # axs[i].fill_between(dates, zmean + zthresh1*zstd, zmean + zthresh2*zstd, alpha=0.1,color='b',label=f'{zthresh2} x Std.Dev')
            # axs[i].fill_between(dates, zmean - zthresh2*zstd, zmean - zthresh1*zstd, alpha=0.1,color='b')
            axs[i].plot(dates,columnData,label=columnName)
            axs[i].plot(dates[mask1], columnData[mask1], 'x')#, label=f'Outliers {zoutliers1+zoutliers2:.2f}%')
            axs[i].set_title(f'{columnName}: {np.sum(mask2)} pts exceed '+ u'\u00B1' + f'{zthresh2} Z')
            axs[i].plot(dates[mask2], columnData[mask2], 'x', color='r')#, label=f'Outliers {zoutliers2:.2f}%')
            axs[i].grid(visible=True,which='Major',axis='both') 
            if Config.MASK_VALUE:
                axs[i].set_yticklabels([])
            # axs[i].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            outlier_stats[columnName]=[zmean,zstd,zoutliers1,zoutliers2,np.sum(mask1),np.sum(mask2),len(columnData),zthresh1,zthresh2]

            i=i+1

# Remove extra ax
for j in range(i,20):
    x,y=fig_indexes(2,j)
    fig.delaxes(axs[j])
fig.set_size_inches(8,i*4)
# fig.subplots_adjust(right=0.7)
fig.tight_layout()
# fig.savefig('./Output Files/Images/Data Exploration/zscore_outliers.png',format='png',bbox_inches='tight')

print(outlier_list)

In [ ]:
outlier_stats={}
outlier_list=[]

zthresh1=3.0
zthresh2=5.0
fig,axs=plt.subplots(1,1,figsize=(8,3))
axs=[axs]

i=0
dates=df_all['Date']

for columnName,columnData in df_all.drop(columns=droplist).select_dtypes(include=np.number).items():
    zmean=np.mean(columnData.loc[lambda x : x != 0])
    zstd=np.std(columnData.loc[lambda x : x != 0])
    zdist = np.abs(columnData - zmean) / zstd
    mask1 = (zthresh1 < zdist) & (zthresh2 > zdist)
    mask2 = (zthresh2 <= zdist)

    if np.sum(mask1) + np.sum(mask2) > 0 and columnName=='VPN_cnxn':
        # x,y=fig_indexes(2,i)
        
        zoutliers1 = 100*np.sum(mask1)/len(columnData)
        zoutliers2 = 100*np.sum(mask2)/len(columnData)
        outlier_stats[columnName]=[zmean,zstd,zoutliers1,zoutliers2,np.sum(mask1),np.sum(mask2),len(columnData),zthresh1,zthresh2]
        if np.sum(mask2) > 0:
            outlier_list.append(columnName)
            # axs[i].hlines(y=zmean,xmin=min(dates),xmax=max(dates), linewidth=0.5, color='r',linestyles='dashed',label='Mean excl. 0 values')
            axs[i].hlines(y=zmean+3*zstd,xmin=min(dates),xmax=max(dates), linewidth=0.5, color='orange',linestyles='dashed',label='')
            axs[i].hlines(y=zmean-5*zstd,xmin=min(dates),xmax=max(dates), linewidth=0.5, color='r',linestyles='dashed',label='')
            # axs[i].fill_between(dates, zmean - zthresh1*zstd, zmean + zthresh1*zstd, alpha=0.1,color='c',label=f'{zthresh1} x Std.Dev')
            # axs[i].fill_between(dates, zmean + zthresh1*zstd, zmean + zthresh2*zstd, alpha=0.1,color='b',label=f'{zthresh2} x Std.Dev')
            # axs[i].fill_between(dates, zmean - zthresh2*zstd, zmean - zthresh1*zstd, alpha=0.1,color='b')
            axs[i].plot(dates,columnData,label='')
            axs[i].plot(dates[mask1], columnData[mask1], 'x', label='> 3Z')#, label=f'Outliers {zoutliers1+zoutliers2:.2f}%')
            axs[i].set_title(f'{columnName}: {np.sum(mask2)} pts exceed '+ u'\u00B1' + f'{zthresh2} Z')
            axs[i].plot(dates[mask2], columnData[mask2], 'x', color='r',label='> 5Z')#, label=f'Outliers {zoutliers2:.2f}%')
            axs[i].grid(visible=True,which='Major',axis='both') 
            if Config.MASK_VALUE:
                axs[i].set_yticklabels([])
            axs[i].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            outlier_stats[columnName]=[zmean,zstd,zoutliers1,zoutliers2,np.sum(mask1),np.sum(mask2),len(columnData),zthresh1,zthresh2]
            axs[i].set_ylabel('VPN_cnxn')
            i=i+1

# Remove extra ax
axs[0].tick_params(axis='x', labelrotation=45)
fig.tight_layout()
fig.savefig('./Output Files/Images/Data Exploration/zscore_outliers.png',format='png',bbox_inches='tight')

# print(outlier_list)

### Export Outlier Statistics

In [ ]:
df_outlier_stats = pd.DataFrame.from_dict(outlier_stats,orient='index',columns=[' mean','std_dev','pct_z_thr1','pct_z_thr2','pts_z_thr1','pts_z_thr2','pts','thresh1','thresh2'])

with engine.connect() as conn:
    df_outlier_stats.reset_index().rename(columns={"index":"date"}).to_sql('OutlierStats',conn,schema='Gold',if_exists='replace',index=False)
    
df_outlier_stats.to_csv('./Output Files/Text/Data Exploration/dimension_reduction_stats.csv')

df_outlier_stats.loc[(df_outlier_stats['pts_z_thr1'] > 0) | (df_outlier_stats['pts_z_thr2'] > 0)].head(100)
df_outlier_stats.head(100)

## Generate Lag Label Values

In [ ]:
# lower = 0.0
# upper = 2.0
# mu = 0.95
# sigma = 0.1

with engine.connect() as conn:
    df_access = pd.read_sql_table('AccessByDirectorate', conn,schema='Bronze')
    df_hols = pd.read_sql_table('HolidayCalendar', conn,schema='Bronze')


df_access=df_access.merge(df_hols[['Date','Public_Holiday']],how='left',on='Date')
df_access=df_access.loc[df_access['Public_Holiday']!=1]

df_access.dropna(subset=['Desks_Used'], inplace=True)
df_access.drop(columns=['AccessByDirectorateID','Day','Public_Holiday'],inplace=True)
df_access['Day_Name']=df_access['Date'].dt.day_name()
df_access['Pct_On_Site']=df_access['Desks_Used']/df_access['Directorate_Numbers']
df_access['Directorate']=df_access['Directorate'].str.strip()


# rangen1=scipy.stats.truncnorm((lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma)
# df_access['Desks_Booked_NoNoise']=df_access['Desks_Booked']
# df_access['Desks_Booked']=df_access['Desks_Booked'].apply(lambda x: np.NaN if np.isnan(x) else int(x*rangen1.rvs(1)[0]))

directorates=df_access['Directorate'].unique()
days=df_access['Day_Name'].unique()

starting_cols = df_access.columns.to_list()

# print(directorates)
per_directorate_per_day=[]
for directorate in directorates:

    df_one_directorate=df_access.loc[df_access['Directorate']==directorate].sort_values('Date')

    for seq_lag in range(1,21):
        df_one_directorate['PctOnSite_seqlag-'+str(seq_lag)]=df_one_directorate['Pct_On_Site'].shift(seq_lag)
        # df_one_directorate['DesksUsed_seqlag-'+str(seq_lag)]=df_one_directorate['Desks_Used'].shift(seq_lag)

    per_day=[]
    for day in days:

        df_one_day = df_one_directorate.loc[df_one_directorate['Day_Name']==day].sort_values('Date')
        for week_lag in range(1,8):
            df_one_day['PctOnSite_SameDaylag-'+str(week_lag)]=df_one_day['Pct_On_Site'].shift(week_lag)
            # df_one_day['DesksUsed_SameDaylag-'+str(week_lag)]=df_one_day['Desks_Used'].shift(week_lag)
            df_one_day.loc[df_one_day.index[1]:,'PctOnSite_SameDaylag-'+str(week_lag)]=df_one_day.loc[df_one_day.index[1]:,'PctOnSite_SameDaylag-'+str(week_lag)].bfill()
            # df_one_day.loc[df_one_day.index[1]:,'DesksUsed_SameDaylag-'+str(week_lag)]=df_one_day.loc[df_one_day.index[1]:,'DesksUsed_SameDaylag-'+str(week_lag)].bfill()
    
        for j,win in enumerate([3,4,5,6,7,8,9,10]):
            df_one_day['PctOnSite_ma'+str(win)]=np.nan
            # df_one_day['DesksUsed_ma'+str(win)]=np.nan
            for pit in range(1,len(df_one_day)):
                ma_pct=np.mean([df_one_day.iloc[x,df_one_day.columns.get_loc('Pct_On_Site')] for x in range(max(0,pit-win),pit)])
                # ma_used=np.mean([df_one_day.iloc[x,df_one_day.columns.get_loc('Desks_Used')] for x in range(max(0,pit-win),pit)])
                df_one_day.iloc[pit,df_one_day.columns.get_loc('PctOnSite_ma'+str(win))]=ma_pct
                # df_one_day.iloc[pit,df_one_day.columns.get_loc('DesksUsed_ma'+str(win))]=ma_used

        per_day.append(df_one_day)


    df_ts_one_directorate= pd.concat(per_day).sort_values(['Date'])
    for day_lag in range(1,8):
        df_ts_one_directorate['PctOnSite_diff-'+str(day_lag)]=df_ts_one_directorate['PctOnSite_seqlag-'+str(day_lag)] - df_ts_one_directorate['PctOnSite_ma5'].shift(day_lag)
    
    per_directorate_per_day.append(df_ts_one_directorate)
        
# print(len(per_directorate_per_day))
df_ts_by_directorate = pd.concat(per_directorate_per_day).sort_values(['Date','Directorate'])
# print(df_ts_by_directorate.info())
pct_cols=[x for x in df_ts_by_directorate.columns.to_list()[len(starting_cols)-1:] if x.startswith('PctOnSite_') ]
# print(pct_cols)
# num_cols=[x for x in df_ts_by_directorate.columns.to_list()[len(starting_cols)-1:] if x.startswith('DesksUsed_') ]




with engine.connect() as conn:
    df_ts_by_directorate[starting_cols+pct_cols].to_sql('Moving_Averages_By_Directorate',conn,schema='Silver',if_exists='replace',index=False,dtype=sqlcol(df_ts_by_directorate))




In [ ]:
fig, axes = joyplot(df_access, by='Directorate', column='Pct_On_Site', colormap=cm.autumn_r, grid="y", overlap = 1, fade=False,title='Attendance Frequency Distribution By Directorate',linewidth=1,figsize=(6,6))
# fig, axes = joyplot(df_access, by='Directorate', column='Pct_On_Site', ylabels=False, xlabels=False, 
#                           grid=False, fill=False, background='k', linecolor="w", linewidth=1,
#                           legend=False, overlap=1.5, figsize=(6,5))

# for a in axes[:-1]:
#     axes[-1].set_xlim([-2,2]) 
axes[-1].set_xlim([0,1]) 

In [ ]:
df_summed_by_directorate=df_access.groupby('Date').sum()
df_summed_by_directorate['Pct_On_Site']=df_summed_by_directorate['Desks_Used']/df_summed_by_directorate['Directorate_Numbers']
df_summed_by_directorate['Day_Name']=df_summed_by_directorate.index.day_name()
df_summed_by_directorate=df_summed_by_directorate[['Day_Name','Pct_On_Site']]

starting_cols = df_summed_by_directorate.columns.to_list()

days=df_summed_by_directorate['Day_Name'].unique()


for seq_lag in range(1,21):
    df_summed_by_directorate['PctOnSite_seqlag-'+str(seq_lag)]=df_summed_by_directorate['Pct_On_Site'].shift(seq_lag)
    df_summed_by_directorate['PctOnSite_seqlag-'+str(seq_lag)]=df_summed_by_directorate['PctOnSite_seqlag-'+str(seq_lag)].bfill()

per_day=[]
for day in days:

    df_one_day = df_summed_by_directorate.loc[df_summed_by_directorate['Day_Name']==day].sort_index()
    

    for week_lag in range(1,8):
        df_one_day['PctOnSite_SameDaylag-'+str(week_lag)]=df_one_day['Pct_On_Site'].shift(week_lag)
        # df_one_day['DesksUsed_SameDaylag-'+str(week_lag)]=df_one_day['Desks_Used'].shift(week_lag)
        df_one_day.loc[df_one_day.index[1]:,'PctOnSite_SameDaylag-'+str(week_lag)]=df_one_day.loc[df_one_day.index[1]:,'PctOnSite_SameDaylag-'+str(week_lag)].bfill()
        # df_one_day.loc[df_one_day.index[1]:,'DesksUsed_SameDaylag-'+str(week_lag)]=df_one_day.loc[df_one_day.index[1]:,'DesksUsed_SameDaylag-'+str(week_lag)].bfill()

    for j,win in enumerate([3,4,5,6,7,8,9,10]):
        df_one_day['PctOnSite_ma'+str(win)]=np.nan
        # df_one_day['DesksUsed_ma'+str(win)]=np.nan
        for pit in range(1,len(df_one_day)):
            ma_pct=np.mean([df_one_day.iloc[x,df_one_day.columns.get_loc('Pct_On_Site')] for x in range(max(0,pit-win),pit)])
            # ma_used=np.mean([df_one_day.iloc[x,df_one_day.columns.get_loc('Desks_Used')] for x in range(max(0,pit-win),pit)])
            df_one_day.iloc[pit,df_one_day.columns.get_loc('PctOnSite_ma'+str(win))]=ma_pct
            # df_one_day.iloc[pit,df_one_day.columns.get_loc('DesksUsed_ma'+str(win))]=ma_used

    per_day.append(df_one_day)

df_ts_by_day = pd.concat(per_day).sort_index()

for day_lag in range(1,8):
    df_ts_by_day['PctOnSite_diff-'+str(day_lag)]=df_ts_by_day['PctOnSite_seqlag-'+str(day_lag)] - df_ts_by_day['PctOnSite_ma5'].shift(day_lag)

pct_cols=[x for x in df_ts_by_day.columns.to_list()[len(starting_cols)-1:] if x.startswith('PctOnSite_') ]
with engine.connect() as conn:
    df_ts_by_day[starting_cols+pct_cols].dropna().drop(columns='Day_Name').to_sql('Moving_Averages_By_Day',conn,schema='Silver',if_exists='replace',dtype=sqlcol(df_ts_by_day))


## Moving Average Prediction Quality by Day

In [ ]:
days=['Monday','Tuesday','Wednesday','Thursday','Friday']
fig,axs=plt.subplots(5,1,figsize=(20,20),sharex=False)

for i,day in enumerate(days):
    #Create a new DF per day
    
    df=df_ts_by_day.loc[df_ts_by_day['Day_Name']==day].dropna()
    
    sns.lineplot(data=df,x='Date',y='Pct_On_Site',label='Actual Attendance',ax=axs[i],linewidth = 2)
    for j,win in enumerate([5,6,7,8,9]):
        col='PctOnSite_ma' + str(win)
        rms = root_mean_squared_error(df['Pct_On_Site'], df[col])
        mae = mean_absolute_error(df['Pct_On_Site'], df[col])
        r2 = r2_score(df['Pct_On_Site'], df[col])
        sns.lineplot(data=df,x='Date',y=col,ax=axs[i],label=f'{str(win)} day MA',linewidth = 1.0,dashes=(2,2))
        

    axs[i].set_title(f'Moving Averages for Attendance on {day}' ,fontsize=12)
    axs[i].set_ylabel('% Staff On-Site',fontsize=10)
    axs[i].set_xlabel('Date',fontsize=10)
    axs[i].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    axs[i].set_ylim(0,0.6)
    axs[i].grid(visible=True,which='Major',axis='both') 
    axs[i].legend(loc='upper left')
    axs[i].legend(bbox_to_anchor=(1.02, 1), loc='upper left')

    if Config.MASK_VALUE:
        axs[i].set_yticklabels([])


axs[i].tick_params(axis='x', labelrotation=45)



fig.tight_layout()
fig.savefig('./Output Files/Images/Data Exploration/MovingAverages.png',format='png',bbox_inches='tight')

In [ ]:
with open('./Output Files/Text/Data Exploration/timeshifted_info.txt', 'w') as f:
    df_ts_by_day.info(buf=f)
    
# df_ts.describe().to_csv('./Output Files/Text/Data Exploration/timeshifted_attendance.csv')
# msno.matrix(df_ts) 

In [ ]:
fig0,ax0=plt.subplots(figsize=(12,10))
args={'annot':False}
plot_corr_heatmap(df=df_ts_by_day.select_dtypes(include=np.number),ax=ax0,**args)
fig0.savefig('.\\Output Files\\Images\\Data Exploration\\Timeseries_Correlation.png',format='png',bbox_inches='tight')

In [ ]:
fig,ax=plt.subplots(figsize=(12,3))  

y=df_all.set_index('Date')[label_field]

_ = ax.plot(y)



tscv = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=10)
TimeSeriesSplit(gap=0, max_train_size=30, n_splits=4, test_size=None)
boundaries=[]
for i, (train_index, test_index) in enumerate(tscv.split(y[:date_val_end])):

    y_train=y.iloc[train_index]
    y_test=y.iloc[test_index]
    boundaries.append(y_test.index[0])
    boundaries.append(y_test.index[-1])
ax.set_ylim(min(y)*1.1, max(y)*1.1)
ax.set_xlim(min(y.index), max(y.index))
for i in range(10):
    ax.vlines(boundaries[i*2], max(y)*1.1, min(y)*1.1, colors='tab:pink', linestyles='solid', label='')
# ax.vlines(boundaries[2], max(y)*1.1, min(y)*1.1, colors='tab:pink', linestyles='solid', label='')
# ax.vlines(boundaries[4], max(y)*1.1, min(y)*1.1, colors='tab:pink', linestyles='solid', label='')
# ax.vlines(boundaries[6], max(y)*1.1, min(y)*1.1, colors='tab:pink', linestyles='solid', label='')
ax.axvspan(date_test_start, y.index[-1],color='c',alpha=0.2,label=f'Hold Out ({len(y[date_test_start:])} pts)')
ax.vlines(date_test_start, max(y)*1.1, min(y)*1.1, colors='c', linestyles='solid', label='')
ax.vlines(y.index[-1], max(y)*1.1, min(y)*1.1, colors='c', linestyles='solid', label='')
ax.axvspan(boundaries[0], boundaries[-1],color='tab:pink',alpha=0.2,label=f'10 * 10pt Cross Validation')
ax.set_ylabel('Daily Attendance')
ax.set_xlabel('Date')
if Config.MASK_VALUE:
    ax.set_yticklabels([])
ax.set_title('Validation and Test Regions')
ax.legend()
fig.tight_layout()
fig.savefig('./Output Files/Images/Data Exploration/TimeSeriesCV.png',format='png',bbox_inches='tight')


## Desks Books versus Actual Attendance

In [ ]:
days=['Thursday','Friday']
fig,axs=plt.subplots(2,1,figsize=(8,5),sharex=True)

for i,day in enumerate(days):
    #Create a new DF per day
    
    df=df_all.loc[df_all['Day']==day][['Date','Actual_Desks_Used','Desks_Booked']].dropna()
    
    sns.lineplot(data=df,x='Date',y='Actual_Desks_Used',label='Actual Attendance',ax=axs[i],linewidth = 2)
    sns.lineplot(data=df,x='Date',y='Desks_Booked',label='Desks Booked',ax=axs[i],linewidth = 2)
    
    if Config.MASK_VALUE:
        axs[i].set_yticklabels([])

    axs[i].set_title(f'Desks Booked Versus Actual Attendance: {day}',fontsize=11)
    axs[i].legend(loc='upper left')
    axs[i].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    # axs[i].tick_params(axis='x', labelrotation=45, labelsize=10)
    axs[i].grid(visible=True,which='Major',axis='both') 

    
    axs[i].set_ylabel('Staff On Site')

axs[1].tick_params(axis='x', labelrotation=45, labelsize=10)
fig.tight_layout()
fig.savefig('./Output Files/Images/Data Exploration/Booked_Versus_Used.png',format='png',bbox_inches='tight')

In [ ]:
metric_rmse=root_mean_squared_error(df_all['Actual_Desks_Used']/df_all['Total_Staff'],df_all['Desks_Booked']/df_all['Total_Staff'])
print(metric_rmse)

In [ ]:
days=['Monday','Tuesday','Wednesday','Thursday','Friday']
fig,axs=plt.subplots(5,1,figsize=(15,10),sharex=True)

for i,day in enumerate(days):
    #Create a new DF per day
    
    df=df_all.loc[df_all['Day']==day][['Date','Actual_Desks_Used','Desks_Booked']].dropna()
    
    sns.lineplot(data=df,x='Date',y='Actual_Desks_Used',label='Actual Attendance',ax=axs[i],linewidth = 2)
    sns.lineplot(data=df,x='Date',y='Desks_Booked',label='Desks Booked',ax=axs[i],linewidth = 2)